<a href="https://colab.research.google.com/github/VIJAYARAGUL362/Disaster-Data-Cleaning-Analysis-with-API-Integration/blob/main/FINAL_YEAR_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING THE LIBRARIES**

In [ ]:
!pip install country_converter
import pandas as pd
import requests,time,re
import country_converter as coco

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.6 MB/s eta 0:00:00


# **IMPORTING THE DATASET**


In [ ]:
dataset = pd.read_excel("dataset.xlsx")

In [ ]:
dataframe = pd.DataFrame(dataset)

# **DATA SELECTION**

In [ ]:
dataframe.drop(["DisNo.","Historic","Classification Key","Disaster Group","Disaster Subgroup","Disaster Subtype","External IDs"],axis=1,inplace = True)

In [ ]:
dataframe.drop(["Event Name","Country","Subregion","Region","Origin","Associated Types","OFDA/BHA Response"],axis=1,inplace=True)

In [ ]:
dataframe.drop(["Appeal","Declaration","AID Contribution ('000 US$)","Magnitude Scale","River Basin","Reconstruction Costs ('000 US$)","Reconstruction Costs, Adjusted ('000 US$)"],axis = 1,inplace=True)

In [ ]:
dataframe.drop(["No. Homeless","Insured Damage ('000 US$)","Insured Damage, Adjusted ('000 US$)","Total Damage ('000 US$)","Total Damage, Adjusted ('000 US$)","Admin Units","Entry Date","Last Update"],axis=1,inplace=True)

# **DATA PRE-PROCESSING**

HANDLING MISSING VALUES

In [ ]:
dataframe.Magnitude.fillna(0,inplace=True)

<ipython-input-8-39e89db91123>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe.Magnitude.fillna(0,inplace=True)


#**FEATURE** **ENGINEERING**


In [ ]:
# finding the month difference
duration_month = dataframe['End Month']-dataframe['Start Month']
# finding the null values and handling the negative values
duration_month = duration_month.apply(lambda x: -(x)*30 if x < 0 else x*30)
duration_month = duration_month.fillna(0)

In [ ]:
# finding the day difference
duration_day = dataframe['End Day']-dataframe['Start Day']
# finding the null values and handling the negative values
duration_day = duration_month.apply(lambda x: -(x) if x < 0 else x)
duration_day = duration_month.fillna(0)

In [ ]:
# calculating the number of days
duration_disaster = duration_day+duration_month

In [ ]:
dataframe['duration'] = duration_disaster

In [ ]:
dataframe.drop(["Start Year","Start Month","Start Day","End Year","End Month","End Day","No. Injured","No. Affected"],inplace = True,axis = 1)

In [ ]:
dataframe['Total Deaths'] = dataframe['Total Deaths'].fillna(dataframe['duration']*1)

In [ ]:
dataframe['Total Affected'] = dataframe['Total Affected'].fillna(dataframe['Total Deaths']*10)

In [ ]:
dataframe['Disaster Type'].unique()

array(['Storm', 'Flood', 'Epidemic', 'Volcanic activity', 'Earthquake',
       'Drought', 'Mass movement (dry)', 'Mass movement (wet)',
       'Wildfire', 'Infestation', 'Extreme temperature', 'Fog', 'Impact',
       'Animal incident', 'Glacial lake outburst flood'], dtype=object)

*BINNING*

In [ ]:
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 0 if ((x == "Flood") | (x == "Glacial lake outburst flood")) else x)

In [ ]:
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 0 if ((x == "Flood") | (x == "Glacial lake outburst flood")) else x)
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 1 if x=="Wildfire" else x)
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 2 if x=="Earthquake" else x)
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 3 if x=="Storm" else x)
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 4 if x == "Drought" else x)

In [ ]:
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 2 if x=="Earthquake" else x)

In [ ]:
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 3 if x=="Storm" else x)

In [ ]:
dataframe['Disaster Type'] = dataframe['Disaster Type'].apply(lambda x: 4 if x == "Drought" else x)

In [ ]:
new_dataframe = dataframe[(dataframe['Disaster Type'] == 0) | (dataframe['Disaster Type'] == 1) | (dataframe['Disaster Type'] == 2) | (dataframe['Disaster Type'] == 3) | (dataframe['Disaster Type'] == 4)]

In [ ]:
new_dataframe

,Disaster Type,ISO,Location,Magnitude,Latitude,Longitude,Total Deaths,Total Affected,CPI,duration
0,3,USA,Galveston (Texas),220.0,NaN,NaN,6000.0,60000.0,2.652223,0.0
1,0,JAM,Saint James,0.0,NaN,NaN,300.0,3000.0,2.652223,0.0
4,2,TUR,"KARS,KARAKURT,KAGIZMAN,DIGOR",5.9,40.3,43.1,140.0,1400.0,2.652223,0.0
5,4,IND,Bengal,0.0,NaN,NaN,1250000.0,12500000.0,2.652223,0.0
6,4,CPV,Countrywide,0.0,NaN,NaN,11000.0,110000.0,2.652223,0.0
...,...,...,...,...,...,...,...,...,...,...
17348,0,THA,"Nakhon Sawan, Phichit, Phitsanulok, Phetchabun...",0.0,NaN,NaN,1.0,1244.0,NaN,0.0
17349,0,DZA,Djelfa + sevral other areas,0.0,NaN,NaN,6.0,100.0,NaN,0.0
17350,3,USA,"Kentucky, Missouri, Virginie",0.0,NaN,NaN,28.0,15108.0,NaN,0.0
17351,4,SOM,"Gedo, Hiran, Bay, Bakool, Mudug, Bari, Toghdhe...",0.0,NaN,NaN,0.0,4400000.0,NaN,0.0


USING API TO REPLACE THE LOCATION VALUES BASED ON THE COUNTRIES ISO

In [ ]:
lists = []
def city_finder(country):
    time.sleep(2)
    url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/countries/{country}"
    headers = {
        "x-rapidapi-key": "",
        "x-rapidapi-host": ""
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return(response.json()["data"]["capital"])
    except requests.exceptions.RequestException:
        print(country,response.text)
    except KeyError:
        print(response.text)
for country in new_dataframe[new_dataframe['Location'].isna()]['ISO'].unique():
    city = city_finder(coco.convert(names=country,to="ISO2"))
    if city is not None:
        df = (new_dataframe['Location'].isna()) & (new_dataframe['ISO'] == country)
        new_dataframe.loc[df,"Location"] = city

In [ ]:
# loading the csv
stage1_dataset = pd.read_csv("stage1_dataset.csv")
new_dataframe = pd.DataFrame(stage1_dataset)

In [ ]:
# Cleaning the location feature
import re
import pandas as pd

def location_cleaner(location_string):
    # 1. Handle non-string types (e.g., NaN, None, numbers) or empty strings
    if not isinstance(location_string, str) or not location_string.strip():
        return location_string

    # Remove leading/trailing whitespace
    processed_string = location_string.strip()

    # Try to extract content from inside parentheses first (e.g., "City (State)")
    # This regex captures whatever is before the first opening parenthesis
    match_paren = re.match(r'^(.*?)\s*\(.*?\)$', processed_string)
    if match_paren:
        # If a match is found, take the group before the parenthesis
        # Recursively call location_cleaner on the extracted part to handle
        # cases like "CityA and CityB (Country)" -> "CityA and CityB" -> "CityA"
        return location_cleaner(match_paren.group(1))

    # Normalize separators: convert " and ", " & ", and potentially " , " to a single comma
    # Using re.sub for more robust replacement of multiple spaces around 'and'/'&'
    processed_string = re.sub(r'\s*and\s*|\s*&\s*|\s*,\s*', ',', processed_string, flags=re.IGNORECASE)

    # Split by comma and take the first part
    if ',' in processed_string:
        city = processed_string.split(',')[0].strip()
    else:
        # If no comma, 'and', or '&' was found, the whole string is the potential city
        city = processed_string.strip()

    # Capitalize the first letter of the city name
    if city: # Ensure city is not empty before capitalizing
        return city.capitalize()

    return None

new_dataframe['Location'] = new_dataframe['Location'].apply(lambda x: location_cleaner(x))

In [ ]:
new_dataframe['Location']

In [ ]:
df = new_dataframe.loc[new_dataframe["Location"].isna()]
df['ISO'].unique()

In [ ]:
for location in new_dataframe['Location']:
    if not isinstance(location,str):
        continue
    # time.sleep(3)
    header = {
        "X-Api-Key":"",
    }
    parameters = {

        "name":location,
    }
    response = requests.get(url="https://api.api-ninjas.com/v1/city",headers=header,params=parameters)
    response_json = response.json()
    print(response.json(),location)
    if len(response_json) != 0:
        lattitude = response_json[0]['latitude']
        longitude = response_json[0]['longitude']
        city = response_json[0]['name']
        # mask = (new_dataframe['ISO'] == country_codes) & (new_dataframe['Location'].isna())
        new_dataframe.loc[new_dataframe['Location']==location,["Latitude","Longitude"]] = [lattitude,longitude]

In [ ]:
# saving the dataset
new_dataframe.to_csv("stage2_dataset.csv",index=False)

In [ ]:
# loading the dataset
stage2_dataset = pd.read_csv("stage2_dataset.csv")
new_dataframe = pd.DataFrame(stage2_dataset)

In [ ]:
# choosing the lattitude and longitude that are not filled and calling api using another account
for location in new_dataframe[new_dataframe['Latitude'].isna()]['Location']:
    if not isinstance(location,str):
        continue
    # time.sleep(3)
    header = {
        "X-Api-Key":"",
    }
    parameters = {

        "name":location,
    }
    response = requests.get(url="https://api.api-ninjas.com/v1/city",headers=header,params=parameters)
    response_json = response.json()
    print(response.json(),location)
    if len(response_json) != 0:
        lattitude = response_json[0]['latitude']
        longitude = response_json[0]['longitude']
        city = response_json[0]['name']
        # mask = new_dataframe['Latitude'].isna() & new_dataframe['Location'] == location
        new_dataframe.loc[new_dataframe['Location']==location,["Latitude","Longitude"]] = [lattitude,longitude]

In [ ]:
# saving the dataset
new_dataframe.to_csv("stage3_dataset.csv")

In [ ]:
#loading the dataset
stage3_dataset = pd.read_csv("stage3_dataset.csv")
new_dataframe = pd.DataFrame(stage3_dataset)

In [ ]:
geo_code_cache = {}

for location in new_dataframe.loc[new_dataframe['Latitude'].isna()]['Location']:
    if not isinstance(location,str):
        continue

    url = "https://us1.locationiq.com/v1/search"
    headers = {
    "accept": "application/json"}

    parameters = {
     "key":"",
    "q":location,
    "format":"json",
     "limit":1,
    }
    if location in geo_code_cache:
        lat,lon = geo_code_cache[location]
        new_dataframe.loc[new_dataframe['Location']==location,["Latitude","Longitude"]] = [lat,lon]
    else:
        try:
            time.sleep(2)
            response = requests.get(url=url,headers=headers,params=parameters)
            print(response.json())
            if len(response.json()) != 0:
                lat = response.json()[0]["lat"]
                lon = response.json()[0]["lon"]
                geo_code_cache[location] = (float(lat), float(lon))
                # mask = (new_dataframe['Location'] == location) & (new_dataframe['Latitude'].isna())
                new_dataframe.loc[new_dataframe['Location']==location,["Latitude","Longitude"]] = [lat,lon]
        except:
            pass

In [ ]:
# saving the dataset
new_dataframe.to_csv("stage4_dataset.csv")

In [ ]:
# loading the dataset
stage4_dataset =pd.read_csv("stage4_dataset.csv")
new_dataframe_4 = pd.DataFrame(stage4_dataset[]

In [ ]:
new_dataframe_4.drop(["Unnamed: 0.1","Unnamed: 0"],inplace=True,axis=1)

In [ ]:
new_dataframe_4.dropna(subset=["Latitude","Location"],inplace=True)

In [ ]:
new_dataframe_4.drop("Total Affected",inplace=True,axis=1)

In [ ]:
new_dataframe_4['Disaster Type'].value_counts()

,count
Disaster Type,
0,5759
3,4536
2,1621
4,745
1,450


In [ ]:
new_dataframe_4.to_csv("stage5_dataset.csv")

In [ ]:
# adding a new_feature date
# Cleaning the location feature
import re
import pandas as pd

def location_cleaner(location_string):
    # 1. Handle non-string types (e.g., NaN, None, numbers) or empty strings
    if not isinstance(location_string, str) or not location_string.strip():
        return location_string

    # Remove leading/trailing whitespace
    processed_string = location_string.strip()

    # Try to extract content from inside parentheses first (e.g., "City (State)")
    # This regex captures whatever is before the first opening parenthesis
    match_paren = re.match(r'^(.*?)\s*\(.*?\)$', processed_string)
    if match_paren:
        # If a match is found, take the group before the parenthesis
        # Recursively call location_cleaner on the extracted part to handle
        # cases like "CityA and CityB (Country)" -> "CityA and CityB" -> "CityA"
        return location_cleaner(match_paren.group(1))

    # Normalize separators: convert " and ", " & ", and potentially " , " to a single comma
    # Using re.sub for more robust replacement of multiple spaces around 'and'/'&'
    processed_string = re.sub(r'\s*and\s*|\s*&\s*|\s*,\s*', ',', processed_string, flags=re.IGNORECASE)

    # Split by comma and take the first part
    if ',' in processed_string:
        city = processed_string.split(',')[0].strip()
    else:
        # If no comma, 'and', or '&' was found, the whole string is the potential city
        city = processed_string.strip()

    # Capitalize the first letter of the city name
    if city: # Ensure city is not empty before capitalizing
        return city.capitalize()

    return None

dataframe['Location'] = dataframe['Location'].apply(lambda x: location_cleaner(x))

In [ ]:
new_dataframe_4['time'] = dataframe.loc[dataframe['Location'].isin(new_dataframe_4['Location'])]['DisNo.'].apply(lambda x: x)

In [ ]:
new_dataframe_4.dropna(subset=["time"],inplace=True)

In [ ]:
new_dataframe_4['time'] = new_dataframe_4['time'].apply(lambda x: x.split("-")[0])

In [ ]:
new_dataframe_4.drop("CPI",inplace=True,axis=1)

In [ ]:
new_dataframe_4['Disaster Type'].value_counts()
new_dataframe_4.columns

Index(['Disaster Type', 'ISO', 'Location', 'Magnitude', 'Latitude',
       'Longitude', 'Total Deaths', 'Total Affected', 'duration', 'time'],
      dtype='object')

In [ ]:
new_dataframe_4.to_csv("final_dataset.csv",index=False)